<a href="https://colab.research.google.com/github/p3bozuric/headshot_generator/blob/main/headshot_generator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference preparation

This is no_code version of inference (usage) of fine-tuned model.

This notebook is build to be run in Google Colab.


## Giving access to Google Drive

Manual approval to Google is mandatory after running next cell.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparing environment for inference

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install git+https://github.com/huggingface/peft

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-840hvib7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-840hvib7
  Resolved https://github.com/huggingface/diffusers.git to commit 28f9d84549c0b1d24ef00d69a4c723f3a11cffb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2772952 sha256=f8eef419b518b61a0629c1c4e16c4e94e71236ff217a6959fb12ca91f4bfe768
  Stored in directory: /tmp/pip-ephem-wheel-cache-dqzcl70h/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-d4xcce_6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-d4xcce_6
  Resolved https://github.com

## Huggingface token preparation (HF_TOKEN)

How to get & prepare HF_TOKEN:
1. Log in to huggingface
2. Create a token here: https://huggingface.co/settings/tokens
3. You need to click on the key icon to the left and place your token there under the name 'HF_TOKEN'.

In [ ]:
import os
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

HF_TOKEN environment variable has been set.


# Inference

## Getting the model up and running

Fill out the form and run the cell.

In [ ]:
import torch
from diffusers import FluxPipeline

#@markdown ---
#@markdown ## **Project Configuration**
#@markdown Basically, copy this information from your tuning notebook.
project_name = 'professional_headshot_generator' # @param {type:"string"}
output_dir = '/content/drive/MyDrive/headshot-generator' # @param {type:"string"}
#@markdown LoRA scale is used to set how much you want to combine your weights into the model. Bigger the number, more adjustment there is.

lora_scale = 1.25 # @param {type:"slider", min:1.0, max:1.5, step:0.01}

if output_dir[-1] == "/":
  total_output_dir = output_dir + project_name
else:
  total_output_dir = output_dir + "/" + project_name

lora_weights = project_name+".safetensors"


device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.float16)

pipe.load_lora_weights(total_output_dir, weight_name=lora_weights)

pipe = pipe.to(device)

pipe.fuse_lora(lora_scale=1.5)

ModuleNotFoundError: No module named 'diffusers'

## Run the model based on parameters

Fill out the form and run the cell to start generating. Be patient and check out results in model_output inside of output_dir directory you set up.

As long as previous cells have ben run once, you can rerun the next one as many times as you want.

In [ ]:
#@markdown ---
#@markdown ## **Image Generation**
#@markdown ### Prompt
#@markdown This is where you write your prompt. Based on this text your image/images will be generated. What matters is for you to use correct instance prompt (trigger word you provided before fine-tuning).
prompt = "professional corporate portrait of pa3k, high-quality studio lighting, neutral gray background, wearing a navy blue suit with a light blue shirt, no tie, confident and friendly expression with a slight smile, looking directly at the camera, head and shoulders framing, sharp focus on the eyes, soft shadows for depth, 4K resolution, professional photography"  # @param {type:"string"}
#@markdown ---

#@markdown ### Guidance Scale
#@markdown Set the guidance scale for image generation. Closer to 0 is more realistic.
guidance_scale = 0.0  # @param {type:"number"}
#@markdown ---

#@markdown ### Image Height
#@markdown Set the height of the generated image in pixels.
height = 1280  # @param {type:"integer"}
#@markdown ---

#@markdown ### Image Width
#@markdown Set the width of the generated image in pixels.
width = 1024  # @param {type:"integer"}
#@markdown ---
#@markdown ### Inference steps & number of generations
#@markdown Number of generated images depends on how many numbers are written bellow.
#@markdown
#@markdown Numbers represent how many inference steps each image needs to have. Feel free to test it out.
#@markdown
#@markdown Bigger number you set, longer your training will take.
#@markdown
#@markdown Separate numbers like in this example: "50, 75, 100". If there is single value: "50".
inf_step = "50, 60, 70, 80, 90, 100"  # @param {type:"string"}
#@markdown ---
#@markdown ### Generation no.
#@markdown Generation number (this number needs to be different with every generataion). Otherwise previously generated images will be overwritten.
no = "0"  # @param {type:"string"}

inf_steps = [int(each) for each in inf_step.split(", ")]

if output_dir[-1] == "/":
  total_output_dir = output_dir + "model_output/"
else:
  total_output_dir = output_dir + "/model_output/"

for i, step in enumerate(inf_steps):
    prompt = """professional corporate portrait of pa3k, high-quality studio lighting, neutral gray background, wearing a navy blue suit with a light blue shirt, no tie, confident and friendly expression with a slight smile, looking directly at the camera, head and shoulders framing, sharp focus on the eyes, soft shadows for depth, 4K resolution, professional photography"""
    out = pipe(
        prompt=prompt,
        guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_inference_steps=step,
        max_sequence_length=256,
    ).images[0]

    # Pohrana generirane slike
    out.save(f"{total_output_dir}no_{no}_step_{step}_{i+1}.png")

print("Done.")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Done.
